If the following folders do not already exist, the program will crash after a while.

In [3]:
%%bash
mkdir forests/rootedFrequentTrees/adult/Results_NoLeafEdgesWithSplitValues
mkdir forests/rootedFrequentTrees/adult/Results_NoLeafEdgesWithSplitValues/leq6

mkdir: cannot create directory ‘forests/rootedFrequentTrees/adult/Results_NoLeafEdgesWithSplitValues’: File exists


In [1]:
#load_ext line_profiler

import csv,operator,sys,os
import numpy as np
import sklearn
import json
import FeatureGenerators.ReadData
import time
from functools import reduce
import warnings; warnings.simplefilter('ignore')  #do not show warnings in the output (like deprecation warnings)

sys.path.append('arch-forest/data/adult/')
sys.path.append('arch-forest/data/bank/')
sys.path.append('arch-forest/data/wine-quality/')
sys.path.append('arch-forest/data/')
sys.path.append('arch-forest/code/')
import trainForest
import Tree
import FeatureGenerators.DecisionSnippetFeatures

from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.estimator_checks import check_estimator
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

dataPath = "arch-forest/data/"
frequentTreesPath = "forests/rootedFrequentTrees/"
resultsPath = "forests/rootedFrequentTrees/"



#import train data
from FeatureGenerators.ReadData import readDataAdult,readDataBank, readWine, readWineTest, readDataSpambase, readDataMagic,readDataCovertype, readDataMnist, readDataSatlog, readDataSensorlessDrive
dataSet='adult'
if (dataSet == 'adult'):
    X_train,Y_train = FeatureGenerators.ReadData.readDataAdult('train')
    X_test,Y_test = FeatureGenerators.ReadData.readDataAdult('test')
if (dataSet == 'spambase'):
    X_train,Y_train = FeatureGenerators.ReadData.readDataSpambase('train')
    X_test,Y_test = FeatureGenerators.ReadData.readDataSpambase('test')        
if (dataSet == 'letter'):
    X_train,Y_train = FeatureGenerators.ReadData.readDataLetter('train')
    X_test,Y_test = FeatureGenerators.ReadData.readDataLetter('test')  
if (dataSet == 'bank'):
    X_train,Y_train = FeatureGenerators.ReadData.readDataBank('train')
    X_test,Y_test = FeatureGenerators.ReadData.readDataBank('test')
if (dataSet == 'magic'):
    X_train,Y_train = FeatureGenerators.ReadData.readDataMagic('train')
    X_test,Y_test = FeatureGenerators.ReadData.readDataMagic('test')
if (dataSet == 'covertype'):
    X_train,Y_train = FeatureGenerators.ReadData.readDataCovertype('train')
    X_test,Y_test = FeatureGenerators.ReadData.readDataCovertype('test') 
if (dataSet == 'mnist'):
    X_train,Y_train = FeatureGenerators.ReadData.readDataMnist('train')
    X_test,Y_test = FeatureGenerators.ReadData.readDataMnist('test')    
if (dataSet == 'satlog'):
    X_train,Y_train = FeatureGenerators.ReadData.readDataSatlog('train')
    X_test,Y_test = FeatureGenerators.ReadData.readDataSatlog('test')
if (dataSet == 'sensorless-drive'):
    X_train,Y_train = FeatureGenerators.ReadData.readDataSensorlessDrive('train')
    X_test,Y_test = FeatureGenerators.ReadData.readDataSensorlessDrive('test')     
if (dataSet == 'wine-quality'):
        X_train,Y_train = FeatureGenerators.ReadData.readWine()
        #X_test,Y_test = ReadData.readWineTest()
print(len(X_test))


# Classification
scoring_function = 'accuracy'
pattern_max_size=6
variant = 'NoLeafEdgesWithSplitValues'

start_time = time.time()
model = GaussianNB()
normalfeatures_nb_cv_score = cross_val_score(model, X_train, Y_train, cv=5, scoring=scoring_function)
normal_time = time.time() - start_time
normal_score = normalfeatures_nb_cv_score.mean()
print('normal: '+str(normal_score))

for rf_depth in (5,10,15,20):
    
    results_list = []
    time_list = []

    start_time = time.time()
    model = DecisionTreeClassifier(max_depth=rf_depth)
    dt_cv_score = cross_val_score(model, X_train, Y_train, cv=5, scoring=scoring_function)
    dt_time = time.time() - start_time
    dt_score = dt_cv_score.mean()
    print('DT: '+str(dt_score))

    start_time = time.time()    
    model = RandomForestClassifier(max_depth=rf_depth, n_estimators=100)
    rf_cv_score = cross_val_score(model, X_train, Y_train, cv=5, scoring=scoring_function)
    rf_time = time.time() - start_time
    rf_score = rf_cv_score.mean()
    print('RF: '+str(rf_score))
    
    
    for frequency in range(2,26):
        
        for pruning in ['','_pruned_with_sigma_0_0','_pruned_with_sigma_0_1','_pruned_with_sigma_0_2','_pruned_with_sigma_0_3']:
        
            start_time = time.time()
            rootedFrequentTrees = "RF_"+str(rf_depth)+pruning+"_t"+str(frequency)
            f = open(frequentTreesPath+dataSet+'/'+variant+'/leq'+str(pattern_max_size)+'/'+rootedFrequentTrees+'.json')
            frequentpatterns = json.load(f)
            f.close()

            #if (frequency < 4):


            dsf = FeatureGenerators.DecisionSnippetFeatures.FrequentSubtreeFeatures(map(lambda x: x['pattern'], frequentpatterns[-100:]))
            #dsf = DecisionSnippetFeatures.FrequentSubtreeFeatures(frequentpatterns)


            fts = dsf.fit_transform(X_train,0)
            fts_test = dsf.fit_transform(X_test,0)
            #print(fts)

            from sklearn.preprocessing import OneHotEncoder
            fts_onehot = OneHotEncoder(n_values=dsf.get_n_values()).fit_transform(fts)
            fts_onehot_test = OneHotEncoder(n_values=dsf.get_n_values()).fit_transform(fts_test)

            
            #model: KNN
            ''' 
            model = KNeighborsClassifier(n_neighbors=25,metric='euclidean')
            model.fit( fts_onehot.toarray(),Y_train)
            y_pred = model.predict(fts_onehot_test.toarray())


            dsf_score = model.score(fts_onehot_test.toarray(),Y_test)
            dsf_time = time.time() - start_time


            print('t'+str(frequency )+pruning+'_DSF: '+str(dsf_score))
            '''

            #model = svm.SVC(kernel='linear',C=1.0)
            '''
            model = LinearSVC()
            model.fit( fts_onehot.toarray(),Y_train)
            y_pred = model.predict(fts_onehot_test.toarray())


            dsf_score = model.score(fts_onehot_test.toarray(),Y_test)
            dsf_time = time.time() - start_time
            print('t'+str(frequency )+'_DSF: '+str(dsf_score))
    '''




            #print(fts_onehot.toarray()[0])
            #print(fts_onehot_test.toarray()[0])

            #model: Log Reg
            '''model = LogisticRegression()
            model.fit( fts_onehot.toarray(),Y_train)
            y_pred = model.predict(fts_onehot_test.toarray())


            dsf_score = model.score(fts_onehot_test.toarray(),Y_test)
            dsf_time = time.time() - start_time
            print('t'+str(frequency )+'_DSF: '+str(dsf_score))
            #for i in range (0,20):
            #   print(Y_test[i])
            #    print(y_pred[i])
            '''

            #y_pred = model.predict(fts_onehot_test.toarray())
            #conf_mat = confusion_matrix(Y_test,y_pred)
            #print(conf_mat)
            #print(classification_report(Y_test,y_pred))


            model = GaussianNB()
            fts_onehot_nb_cv_score = cross_val_score(model, fts_onehot.toarray(), Y_train, cv=5, scoring=scoring_function)
            dsf_time = time.time() - start_time
            print(fts_onehot_nb_cv_score)
            dsf_score = fts_onehot_nb_cv_score.mean()
            print('t'+str(frequency )+'_DSF: '+str(dsf_score))



            result = "t"+str(frequency)+pruning+","+str(dsf_score)+","+str(normal_score)+","+str(dt_score)+","+str(rf_score)+",\n"
            results_list.append(result)
            times = "t"+str(frequency)+pruning+","+str(dsf_time)+","+str(normal_time)+","+str(dt_time)+","+str(rf_time)+",\n"
            time_list.append(times)

    #f= open(resultsPath+dataSet+'/Results_'+variant+'/leq'+str(pattern_max_size)+'/'+'RF_'+str(rf_depth)+'_'+scoring_function+'_KNN.csv',"a")
    f= open(resultsPath+dataSet+'/Results_'+variant+'/leq'+str(pattern_max_size)+'/'+'RF_'+str(rf_depth)+'_'+scoring_function+'.csv',"a")
    for line in results_list:
        f.write(line)
    f.close()

    #f= open(resultsPath+dataSet+'/Results_'+variant+'/leq'+str(pattern_max_size)+'/'+'RF_'+str(rf_depth)+'_'+scoring_function+'_time_KNN.csv',"a")
    f= open(resultsPath+dataSet+'/Results_'+variant+'/leq'+str(pattern_max_size)+'/'+'RF_'+str(rf_depth)+'_'+scoring_function+'_time.csv',"a")
    for line in time_list:
        f.write(line)
    f.close()
    

7991
normal: 0.635781324184177
DT: 0.8087804488592166
RF: 0.7904976206935481
[0.7743239  0.76958955 0.7761194  0.77549751 0.77860697]
t2_DSF: 0.7748274674188894
[0.76375505 0.76150498 0.76399254 0.75652985 0.76958955]
t2_DSF: 0.7630743933425814
[0.73733292 0.73756219 0.7403607  0.73849502 0.74471393]
t2_DSF: 0.7396929519329063
[0.77960833 0.77643035 0.77767413 0.78482587 0.78575871]
t2_DSF: 0.7808594770938593
[0.76188996 0.77238806 0.77052239 0.77363184 0.77145522]
t2_DSF: 0.7699774944054981
[0.75784893 0.76430348 0.77052239 0.75995025 0.76616915]
t3_DSF: 0.7637588402408226
[0.75349705 0.75435323 0.75995025 0.75124378 0.76150498]
t3_DSF: 0.7561098571488223
[0.73764377 0.75652985 0.75373134 0.75466418 0.75621891]
t3_DSF: 0.7517576092184399
[0.77214796 0.76896766 0.77643035 0.78016169 0.78389303]
t3_DSF: 0.7763201400519936
[0.74417159 0.74844527 0.74720149 0.74533582 0.75031095]
t3_DSF: 0.747093024154948
[0.78116257 0.78575871 0.7829602  0.78327114 0.78358209]
t4_DSF: 0.7833469426260058


[0.60366801 0.60074627 0.60478856 0.60323383 0.60603234]
t21_DSF: 0.603693801740443
[0.60366801 0.60074627 0.60478856 0.60323383 0.60603234]
t21_DSF: 0.603693801740443
[0.76562014 0.75279851 0.74595771 0.75279851 0.75217662]
t22_DSF: 0.7538702972547892
[0.76562014 0.75279851 0.74595771 0.75279851 0.75217662]
t22_DSF: 0.7538702972547892
[0.59900528 0.59732587 0.59577114 0.59794776 0.59856965]
t22_DSF: 0.5977239424574361
[0.60366801 0.60074627 0.60478856 0.60323383 0.60603234]
t22_DSF: 0.603693801740443
[0.51227852 0.51243781 0.51243781 0.51243781 0.51243781]
t22_DSF: 0.5124059528283358
[0.51227852 0.51243781 0.51243781 0.51243781 0.51243781]
t23_DSF: 0.5124059528283358
[0.51227852 0.51243781 0.51243781 0.51243781 0.51243781]
t23_DSF: 0.5124059528283358
[0.59900528 0.59732587 0.59577114 0.59794776 0.59856965]
t23_DSF: 0.5977239424574361
[0.51227852 0.51243781 0.51243781 0.51243781 0.51243781]
t23_DSF: 0.5124059528283358
[0.51227852 0.51243781 0.51243781 0.51243781 0.51243781]
t23_DSF: 0.

[0.81100404 0.80628109 0.80721393 0.81281095 0.8159204 ]
t17_DSF: 0.8106460818382443
[0.79110973 0.78358209 0.79570896 0.79011194 0.80037313]
t17_DSF: 0.7921771697929377
[0.78955549 0.7925995  0.78109453 0.79073383 0.79726368]
t17_DSF: 0.7902494057533285
[0.78427106 0.78420398 0.77767413 0.7863806  0.79166667]
t17_DSF: 0.7848392866256223
[0.79577246 0.78793532 0.78264925 0.79477612 0.79912935]
t17_DSF: 0.7920525017127604
[0.80758471 0.80721393 0.8028607  0.80876866 0.81467662]
t18_DSF: 0.808220921349114
[0.79888094 0.7925995  0.79322139 0.80597015 0.81560945]
t18_DSF: 0.8012562884984465
[0.796705   0.79104478 0.78513682 0.7925995  0.79850746]
t18_DSF: 0.792798712375332
[0.78551445 0.78824627 0.77487562 0.78358209 0.79477612]
t18_DSF: 0.785398910792633
[0.79328567 0.78700249 0.77985075 0.79135572 0.79819652]
t18_DSF: 0.7899382285031169
[0.80447622 0.80441542 0.79477612 0.80659204 0.80690299]
t19_DSF: 0.803432557449
[0.79079888 0.78047264 0.79197761 0.79166667 0.79508706]
t19_DSF: 0.7900

[0.75349705 0.74782338 0.74502488 0.74004975 0.74813433]
t12_DSF: 0.7469058770493198
[0.73422443 0.73041045 0.73662935 0.73041045 0.73880597]
t12_DSF: 0.7340961303213493
[0.7416848  0.75031095 0.74502488 0.75031095 0.75746269]
t12_DSF: 0.7489588504477922
[0.76935033 0.75466418 0.75746269 0.7596393  0.76741294]
t13_DSF: 0.7617058861737319
[0.76064657 0.74378109 0.74937811 0.75435323 0.75901741]
t13_DSF: 0.7534352831738108
[0.75225365 0.74409204 0.74067164 0.7403607  0.75310945]
t13_DSF: 0.7460974966634035
[0.75815978 0.74813433 0.75901741 0.75124378 0.76181592]
t13_DSF: 0.7556742437950131
[0.73733292 0.73258706 0.73911692 0.73662935 0.73756219]
t13_DSF: 0.7366456882513142
[0.76624184 0.75621891 0.76337065 0.76181592 0.76865672]
t14_DSF: 0.7632608058557074
[0.77276966 0.75746269 0.7568408  0.76243781 0.76896766]
t14_DSF: 0.7636957232797776
[0.75225365 0.74253731 0.73849502 0.74222637 0.7534204 ]
t14_DSF: 0.7457865513897717
[0.76220081 0.7534204  0.75870647 0.7568408  0.76616915]
t14_DSF:

[0.72831831 0.73289801 0.73476368 0.73880597 0.7300995 ]
t7_DSF: 0.7329770946325259
[0.72894001 0.73538557 0.72450249 0.73134328 0.73942786]
t8_DSF: 0.7319198420394144
[0.74323904 0.74502488 0.74129353 0.75621891 0.75932836]
t8_DSF: 0.7490209428456104
[0.75101026 0.75310945 0.74875622 0.76679104 0.76710199]
t8_DSF: 0.7573537928944026
[0.64811937 0.65609453 0.64707711 0.64583333 0.64210199]
t8_DSF: 0.6478452662085903
[0.64687597 0.65329602 0.65298507 0.65671642 0.64894279]
t8_DSF: 0.6517632539818781
[0.70811315 0.710199   0.70833333 0.72014925 0.70771144]
t9_DSF: 0.7109012367444716
[0.7199254  0.73165423 0.7170398  0.72947761 0.72947761]
t9_DSF: 0.725514930012666
[0.72925085 0.73787313 0.71828358 0.72170398 0.73631841]
t9_DSF: 0.7286859918622616
[0.65806652 0.65267413 0.64552239 0.64552239 0.64085821]
t9_DSF: 0.6485287272063679
[0.7180603  0.71921642 0.70864428 0.71859453 0.70242537]
t9_DSF: 0.713388180329314
[0.70127448 0.69869403 0.70553483 0.70833333 0.69589552]
t10_DSF: 0.7019464381